# EDA

In this notebook the raw text data is cleaned and word embeddings are generated with an LSTM.

**Data Sources:**

- `data/raw/train.csv`: labelled data for approx. 7'600 tweets
- `data/raw/test.csv`: test data for approx. 3'400 tweets

**Data Output:**

- `xxx.csv`: blablabla

**Changes**

- 2020-01-07: Start notebook, complete preprocessing

___

_**Open Questions / Things to check out for the future:**_
- Import pre-trained embedding (FastText, GloVe), see this [MLM post](https://machinelearningmastery.com/develop-word-embeddings-python-gensim/) and the appendix
- Preprocessing with removal of stop-words and / or punctuation

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries,-load-data" data-toc-modified-id="Import-libraries,-load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries, load data</a></span></li><li><span><a href="#Preprocess-text" data-toc-modified-id="Preprocess-text-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocess text</a></span><ul class="toc-item"><li><span><a href="#Clean" data-toc-modified-id="Clean-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Clean</a></span></li><li><span><a href="#Encode" data-toc-modified-id="Encode-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Encode</a></span></li><li><span><a href="#Subsample" data-toc-modified-id="Subsample-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Subsample</a></span></li><li><span><a href="#Create-batches¶" data-toc-modified-id="Create-batches¶-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Create batches¶</a></span></li></ul></li><li><span><a href="#Build-NN-for-Word2Vec-Embedding" data-toc-modified-id="Build-NN-for-Word2Vec-Embedding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Build NN for Word2Vec Embedding</a></span><ul class="toc-item"><li><span><a href="#Define-Model-Architecture-and-Loss-Function" data-toc-modified-id="Define-Model-Architecture-and-Loss-Function-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Define Model Architecture and Loss Function</a></span></li><li><span><a href="#Define-validation-function" data-toc-modified-id="Define-validation-function-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Define validation function</a></span></li></ul></li><li><span><a href="#Training-NN" data-toc-modified-id="Training-NN-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training NN</a></span></li><li><span><a href="#4.-Embeddings-&amp;-Text-Cleaning" data-toc-modified-id="4.-Embeddings-&amp;-Text-Cleaning-5"><span class="toc-item-num">5&nbsp;&nbsp;</span><strong>4. Embeddings &amp; Text Cleaning</strong></a></span><ul class="toc-item"><li><span><a href="#4.1-Embedding-Coverage" data-toc-modified-id="4.1-Embedding-Coverage-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span><strong>4.1 Embedding Coverage</strong></a></span></li><li><span><a href="#4.2-Text-Cleaning-(Not-Finished)" data-toc-modified-id="4.2-Text-Cleaning-(Not-Finished)-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span><strong>4.2 Text Cleaning (Not Finished)</strong></a></span></li></ul></li><li><span><a href="#5.-Test-Set-Labels" data-toc-modified-id="5.-Test-Set-Labels-6"><span class="toc-item-num">6&nbsp;&nbsp;</span><strong>5. Test Set Labels</strong></a></span></li></ul></div>

In [1]:
# import gc
# import re
# import string
# import operator
# from collections import defaultdict


# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

# import matplotlib.pyplot as plt
# import seaborn as sns

# from wordcloud import STOPWORDS
# import tensorflow_hub as hub

SEED = 42

## Import libraries, load data

In [39]:
from collections import Counter
from string import punctuation
from random import sample, random

import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.optim as optim

# Visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set_style('whitegrid')
col1, col2 = 'rebeccapurple', 'yellow'

# Display settings
from IPython.display import display
pd.options.display.max_columns = 100

In [3]:
df_train = pd.read_csv('data/raw/train.csv', dtype={'id': np.int16, 'target': np.int8})
df_test = pd.read_csv('data/raw/test.csv', dtype={'id': np.int16})

print(f'Training Set Shape = {df_train.shape}')
print(f'Training Set Memory Usage = {df_train.memory_usage().sum() / 1024**2:.2f} MB')
print(f'Test Set Shape = {df_test.shape}')
print(f'Test Set Memory Usage = {df_test.memory_usage().sum() / 1024**2:.2f} MB')

Training Set Shape = (7613, 5)
Training Set Memory Usage = 0.20 MB
Test Set Shape = (3263, 4)
Test Set Memory Usage = 0.08 MB


In [4]:
display(df_train.head(2))
display(df_test.head(2))

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."


## Preprocess text

### Clean

In [5]:
# Define cleaning function

def clean_text(text_list, trim_threshold=5):
    """Clean the input text (replace punctuation, remove unfrequent words) 
    Return cleaned text as list of tweets and as one large string.
    
    Arguments:
    ----------
    - text_list: iterable of strings
    - trim_threshold: int, all words with <= this frequency will be removed
    
    Returns:
    --------
    - text_complete: str, cleaned text 
    - text_list: list of strings, cleaned text split into individual tweets
    """
    
    # Concatenate tweets into one large string for further cleaning
    text = ""
    for item in text_list:
        text = text + item + " || "
    
    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')  
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_Double> ')
    text = text.replace("'", ' <QUOTATION> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
#     text = text.replace('\n', ' <NEW_LINE> ')  # watch whitespaces
    text = text.replace(':', ' <COLON> ')
    text = text.replace('#', ' <HASHTAG> ')
    text = text.replace('@', ' <MENTION> ')
    
      
    # Remove all words with x or fewer occurences
    words_all = text.split(" ")
    word_counts = Counter(words_all)
    words_trimmed = [word for word in words_all if word_counts[word] > trim_threshold]
    text_trimmed = ' '.join([word for word in words_trimmed])

    # Split by new lines, reassemble 
    text_list = text_trimmed.split(' || ')[:-1] # last item is empty
    text_complete = ' '.join(text_list)  

    return text_complete, text_list

In [6]:
text_complete, tweets_train = clean_text(df_train['text'], 3)

vocab_list = list(set(text_complete.split()))
print(f"Number of tweets: {len(tweets_train)}")
print(f"Number of unique words in text_complete: {len(vocab_list)}")

Number of tweets: 7613
Number of unique words in text_complete: 3489


### Encode
Create 2 dicts mapping words in vocabulary to integers and vice-versa.
-  sure that most frequent words get lowest int representation
- DON'T leave pos [0] for padding (see next section) yet

In [7]:
def encode_text(text, start_pos=0):
    """Encode words to integers (and vice versa), so that most frequent words
    get lowest int representations. Return two mapping dictionaries.
    
    Arguments:
    ----------
    - text: string, input text with words delimited by whitespaces
    - start_pos: int, value for first integer representation (default=0)
    
    Returns:
    --------
    - vocab_to_int: dict, mapping of words to ints (most frequent comes first)
    - int_to_vocab: dict, mapping of ints to words
    """

    word_count_dict = Counter(text.split())
    vocab = sorted(word_count_dict, key=word_count_dict.get, reverse=True)
    vocab_to_int = {word: i for i, word in enumerate(vocab, start_pos)}
    int_to_vocab = {i: word for i, word in enumerate(vocab, start_pos)}

    return vocab_to_int, int_to_vocab

In [8]:
vocab_to_int, int_to_vocab = encode_text(text_complete, start_pos=0)

In [9]:
# Check results
print('Unique words: ', len((vocab_to_int)))
assert len(vocab_to_int) == len(vocab_list)
print(list(vocab_to_int.items())[:7])
print(list(int_to_vocab.items())[:7])

Unique words:  3489
[('<PERIOD>', 0), ('<COLON>', 1), ('//t', 2), ('http', 3), ('<HASHTAG>', 4), ('the', 5), ('<QUOTATION>', 6)]
[(0, '<PERIOD>'), (1, '<COLON>'), (2, '//t'), (3, 'http'), (4, '<HASHTAG>'), (5, 'the'), (6, '<QUOTATION>')]


### Subsample
Discard some high-frequent words from our data and in return get faster training and better representations (noise reduction). (According to [Neural Information Processing Systems, paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf), Mikolov et al.)



In [10]:
def subsample(text, vocab_to_int, threshold=1e-5):
    """Discard some frequent words, according to the subsampling equation.
    Return a new (reduced) list of words for training. 
    
    Arguments:
    ----------
    - text: string, dict, mapping of words to ints (most frequent come first)
    - vocab_to_int: int, value for first integer representation (default=0)
    - threshold: float, threshold for 'random discarding' a word (default=1e-5)
    
    Returns:
    --------
    - words_train: list, subsampled text represented in ints instead of words
    """ 
    
    # Encode the whole text into a list of ints using a mapping dict
    int_words = [vocab_to_int[word] for word in text.split()]
    # Create dictionary of int_words, showing their frequencies
    word_counts = Counter(int_words)  
    
    total_count = len(int_words)
    freqs = {word: count / total_count for word, count in word_counts.items()}
    p_drop = {word: 1 - np.sqrt(threshold / freqs[word]) for word in word_counts}

    words_train = [word for word in int_words if random() < (1 - p_drop[word])]
    
    return words_train

In [11]:
words_train = subsample(text_complete, vocab_to_int)

In [12]:
# Check results
print("Length train text:", len(words_train))
print("Length initial text:", len(text_complete.split()))
print(words_train[:10])

Length train text: 16341
Length initial text: 134102
[32, 258, 199, 138, 2107, 668, 1374, 1085, 259, 1195]


### Create batches¶ 
"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose , for each training word we will select randomly a number  in range , and then use  words from history and  words from the future of the current word as correct labels."
(According to original [Word2Vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.)

In [13]:
# This function will be called within the next function

def get_target(words, idx, window_size=5):
    """ Get a random-length list of target words (ints) in a window around 
    an input index (the input word).
    
    Arguments:
    ----------
    - words: list, text represented as list of words / ints
    - idx: int, the index of the input word in words
    - window_size: max window to and from idx to get target words
    
    Returns:
    --------
    - target: list, of words near the idx, the 'label' four our input
    
    """
    
    R = np.random.randint(1, window_size+1)
    min_val = np.max([idx-R, 0])  # make sure no neg index occurs, not necessary for values at end
    target = words[min_val : idx] + words[idx+1 : idx+R+1]
    
    return target

In [14]:
def get_batches(words, batch_size, window_size=5):
    """Create a generator of word batches as a tuple (inputs, targets). It
    grabs `batch_size` words from a words list. Then for each of those 
    batches, it gets the target words in a window.
    
    Arguments:
    ----------
    - words: list, text represented as list of words / ints
    - batch size: int, number of inputs to form one batch
    - window_size: max window to and from idx to get target words
    
    Returns:
    --------
    - x, y: lists, inputs und corresponding labels for one batch at a time
    
    """
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx : idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)  # each batch x and y will be one row of values
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [15]:
# Check results
test_text = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
x, y = next(get_batches(test_text, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D']
y
 ['B', 'C', 'D', 'A', 'C', 'D', 'A', 'B', 'D', 'A', 'B', 'C']


## Build NN for Word2Vec Embedding

### Define Model Architecture and Loss Function

In [29]:
class SkipGramNeg(nn.Module):
    def __init__(self, n_vocab, n_embed, noise_dist=None):
        super().__init__()
        
        self.n_vocab = n_vocab
        self.n_embed = n_embed
        self.noise_dist = noise_dist
        
        # Define embedding layers for input and output words
        self.in_embed = nn.Embedding(n_vocab, n_embed)
        self.out_embed = nn.Embedding(n_vocab, n_embed)
        
        # Initialize both embedding tables with uniform distribution
        # (this may help with convergence)
        self.in_embed.weight.data.uniform_(-1, 1)
        self.out_embed.weight.data.uniform_(-1, 1)
        
    def forward_input(self, input_words):
        input_vectors = self.in_embed(input_words)
        return input_vectors
    
    def forward_output(self, output_words):
        output_vectors = self.out_embed(output_words)
        return output_vectors
    
    def forward_noise(self, batch_size, n_samples):
        """ Generate noise vectors with shape (batch_size, n_samples, n_embed)"""
        if self.noise_dist is None:
            # Sample words uniformly
            noise_dist = torch.ones(self.n_vocab)
        else:
            noise_dist = self.noise_dist
            
        # Sample words from our noise distribution
        noise_words = torch.multinomial(noise_dist,
                                        batch_size * n_samples,
                                        replacement=True)
        
        device = "cuda" if self.out_embed.weight.is_cuda else "cpu"
        noise_words = noise_words.to(device)
        noise_vectors = self.out_embed(noise_words).view(batch_size, n_samples, self.n_embed)      
        
        return noise_vectors

In [30]:
class NegativeSamplingLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input_vectors, output_vectors, noise_vectors):
        
        batch_size, embed_size = input_vectors.shape
        
        # Input vectors should be a batch of column vectors
        input_vectors = input_vectors.view(batch_size, embed_size, 1)
        
        # Output vectors should be a batch of row vectors
        output_vectors = output_vectors.view(batch_size, 1, embed_size)
        
        # bmm = batch matrix multiplication
        # Correct log-sigmoid loss
        out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log()
        out_loss = out_loss.squeeze()
        
        # Incorrect log-sigmoid loss
        noise_loss = torch.bmm(noise_vectors.neg(), input_vectors).sigmoid().log()
        noise_loss = noise_loss.squeeze().sum(1)  # sum the losses over the sample of noise vectors

        # Negate and sum correct and noisy log-sigmoid losses
        # Return average batch loss
        return -(out_loss + noise_loss).mean()

### Define validation function
This function helps observing the model during training. It will print out the closest words to some input words using the _cosine similarity_. We will input a mix of a few common words and a few uncommon words.

We can encode the validation words as vectors using the embedding table, then calculate the similarity with each word vector  in the embedding table. With the similarities, we can print out the validation words and words in our embedding table semantically similar to those words.

In [40]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the 
    embedding matrix.
    
    Arguments:
    ----------
    - embedding: a PyTorch embedding module
    - ...    
    """
    
    # Calculate the cosine similarity between some random words and the embedding vectors. 
    # With the similarities, we can look at what words are close to our random words.
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # Calculate magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # Pick N words from our ranges (0, window) and (1000, 1000+window). 
    # lower id implies more frequent words, higher id more uncommon words
    valid_examples = np.array(sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               sample(range(1000, 1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t()) / magnitudes
        
    return valid_examples, similarities

## Training NN
(Training on GPU recommended, if available.)

In [32]:
# Check for a GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if not torch.cuda.is_available():
    print('No GPU found. Please use a GPU to train your neural network.')

No GPU found. Please use a GPU to train your neural network.


In [33]:
def train_w2v(model, words, batch_size, optimizer, criterion, epochs, print_every=1500):
    """Train loop with forward and backward propagation, """
    
    model.train()
    steps = 0
    for e in range(epochs):

        # Get our input, target batches
        for input_words, target_words in get_batches(words, batch_size):
            steps += 1
            inputs, targets = torch.LongTensor(input_words), \
                              torch.LongTensor(target_words)
            inputs, targets = inputs.to(device), targets.to(device)

            # input, outpt, and noise vectors
            input_vectors = model.forward_input(inputs)
            output_vectors = model.forward_output(targets)
            noise_vectors = model.forward_noise(inputs.shape[0], 5)

            # negative sampling loss
            loss = criterion(input_vectors, output_vectors, noise_vectors)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # loss stats
            if steps % print_every == 0:
                print("Epoch: {}/{}".format(e + 1, epochs))
                print("Loss: ", loss.item()) # avg batch loss at this point
                valid_examples, valid_similarities = cosine_similarity(
                        model.in_embed, device=device)
                _, closest_idxs = valid_similarities.topk(6)

                valid_examples, closest_idxs = valid_examples.to('cpu'), \
                                               closest_idxs.to('cpu')
                for ii, valid_idx in enumerate(valid_examples):
                    closest_words = [int_to_vocab[idx.item()] \
                                     for idx in closest_idxs[ii]][1:]
                    print(int_to_vocab[valid_idx.item()] \
                          + " | " + ', '.join(closest_words))
                print("...\n")
    
    # Return trained model
    return model

In [34]:
# Get noise distribution

# Calculate new freqs after subsampling - this was not done in orig Udacity project!
word_counts = Counter(words_train)
total_count = len(words_train)
freqs = {word: count / total_count for word, count in word_counts.items()}
word_freqs = np.array(sorted(freqs.values(), reverse=True))
unigram_dist = word_freqs / word_freqs.sum()
noise_dist = torch.from_numpy(unigram_dist**(0.75)/np.sum(unigram_dist**(0.75)))

In [42]:
# Set net parameters
embedding_dim = 300

# Instantiate model and move to GPU if available
w2v_net = SkipGramNeg(len(vocab_to_int), embedding_dim, noise_dist=noise_dist)
w2v_net.to(device)

# Set training parameters
print_every = 500
batch_size = 64
epochs = 20

criterion = NegativeSamplingLoss() 
optimizer = optim.Adam(w2v_net.parameters(), lr=0.003)

In [43]:
# Train model
trained_w2v_net = train_w2v(w2v_net, words_train, batch_size, 
                            optimizer, criterion, epochs, print_every)

Epoch: 2/20
Loss:  11.532182693481445
don | plunging, marvel, meek, decide, english
are | tuesday, anthrax, beû_, about, movie
s | ban, fedex, shoes, 45, nu
fire | 10th, bowl, reason, 4000, santa
<EXCLAMATION_MARK> | anyone, bank, mind, ship, alberta
one | entertainment, bell, two, christian, these
from | black, forces, always, tcot, theory
http | i, explosion, lucky, tube, capsizes
wonder | captures, breaking, ferguson, radiation, okwx
nigerian | been, careful, aba, co/moll5vd8yd, bed
landing | week, fallen, latest, google, speaker
share | catastrophe, kyle, nice, nothing, dick
dad | gun, thru, trial, our, shares
told | talk, california, reddit, mansehra, drill
early | ross, civilians, details, earth, usually
tried | terror, m1, that, sees, forced
...

Epoch: 4/20
Loss:  8.58129596710205
http | i, send, explosion, lucky, have
me | animal, been, in], footage, wedding
the | wearing, on, tons, exploration, but
i | http, egypt, giving, target, predicted
body | wins, memories, clearly, tr

In [45]:
torch.save(w2v_net.state_dict(), 'models/w2v_net.pth')

#  state_dict = torch.load('checkpoint.pth')

## **4. Embeddings & Text Cleaning**

### **4.1 Embedding Coverage**
* When you have pre-trained embeddings, doing standard preprocessing steps  might not be a good idea because some of the valuable information can be lost. It is better to get vocabulary as close to embeddings as possible. In order to do that, `train_vocab` and `test_vocab` are created by counting the words in tweets.

In [ ]:
def build_vocab(tweets):
    vocab = {}        
    for tweet in tweets:
        for word in tweet:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

train_tweets = df_train['text'].apply(lambda s: s.split()).values
train_vocab = build_vocab(train_tweets)
test_tweets = df_test['text'].apply(lambda s: s.split()).values
test_vocab = build_vocab(test_tweets)

Embeddings used for text cleaning are:
* GloVe-300d-840B
* FastText-Crawl-300d-2M

In [ ]:
embeddings_glove = np.load('../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl', allow_pickle=True)
embeddings_fasttext = np.load('../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl', allow_pickle=True)

Words in intersection of vocab and embeddings are stored in `covered` along with their counts. Words in vocab that don't exist in embeddings are stored in `oov` along with their counts. `n_covered` and `n_oov` are total number of counts and they are used for calculating coverage percentages.

Both GloVe and FastText embeddings have more than **50%** vocab and **80%** text coverage without cleaning. GloVe and FastText coverage are very close but GloVe has slightly higher coverage.

In [ ]:
def check_coverage(vocab, embeddings, embeddings_name, dataset_name):
    covered = {}
    oov = {}    
    n_covered = 0
    n_oov = 0
    
    for word in vocab:
        try:
            covered[word] = embeddings[word]
            n_covered += vocab[word]
        except:
            oov[word] = vocab[word]
            n_oov += vocab[word]
            
    vocab_coverage = len(covered) / len(vocab)
    text_coverage = (n_covered / (n_covered + n_oov))
    print('{} Embeddings cover {:.2%} of {} vocab'.format(embeddings_name, vocab_coverage, dataset_name))
    print('{} Embeddings cover {:.2%} of {} text'.format(embeddings_name, text_coverage, dataset_name))
    
    sorted_oov = sorted(oov.items(), key=operator.itemgetter(1))[::-1]
    return sorted_oov

train_oov_glove = check_coverage(train_vocab, embeddings_glove, 'GloVe', 'Training')
test_oov_glove = check_coverage(test_vocab, embeddings_glove, 'GloVe', 'Test')
train_oov_fasttext = check_coverage(train_vocab, embeddings_fasttext, 'FastText', 'Training')
test_oov_fasttext = check_coverage(test_vocab, embeddings_fasttext, 'FastText', 'Test')

### **4.2 Text Cleaning (Not Finished)**
* The most common type of words in oov have punctuations at the start or end. Those words doesn't have embeddings because of those trailing punctuations. `#@!?()[]*%...'` are separated from words.
* There are special characters in tweets and some of them are attached to words. They are removed completely.
* Contractions are expanded.
* Character entity references are replaced with their actual symbols.
* Typos and slang are corrected, and informal abbreviations are written in their long forms.
* Hashtags and usernames are expanded.

In [ ]:
def clean(tweet):    
    # Punctuations at the start or end of words    
    for punctuation in "#@!?()[]*%":
        tweet = tweet.replace(punctuation, f' {punctuation} ').strip()
    tweet = tweet.replace('...', ' ... ').strip()
    tweet = tweet.replace("'", " ' ").strip()        
    
    # Special characters
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"China\x89Ûªs", "China's", tweet)
    tweet = re.sub(r"let\x89Ûªs", "let's", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
    
    # Contractions
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "Here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    
    # Character entity references
    tweet = re.sub(r"&gt;", ">", tweet)
    tweet = re.sub(r"&lt;", "<", tweet)
    tweet = re.sub(r"&amp;", "&", tweet)
        
    # Typos, slang and informal abbreviations
    tweet = re.sub(r"w/e", "whatever", tweet)
    tweet = re.sub(r"w/", "with", tweet)
    tweet = re.sub(r"USAgov", "USA government", tweet)
    tweet = re.sub(r"recentlu", "recently", tweet)
    tweet = re.sub(r"Ph0tos", "Photos", tweet)
    tweet = re.sub(r"exp0sed", "exposed", tweet)
    tweet = re.sub(r"<3", "love", tweet)
        
    # Hashtags and usernames
    tweet = re.sub(r"IranDeal", "Iran Deal", tweet)
    tweet = re.sub(r"ArianaGrande", "Ariana Grande", tweet)
    tweet = re.sub(r"camilacabello97", "camila cabello", tweet) 
    tweet = re.sub(r"RondaRousey", "Ronda Rousey", tweet)     
    tweet = re.sub(r"MTVHottest", "MTV Hottest", tweet)
    tweet = re.sub(r"TrapMusic", "Trap Music", tweet)
    tweet = re.sub(r"ProphetMuhammad", "Prophet Muhammad", tweet)
    tweet = re.sub(r"PantherAttack", "Panther Attack", tweet)
    tweet = re.sub(r"StrategicPatience", "Strategic Patience", tweet)
    tweet = re.sub(r"socialnews", "social news", tweet)
    tweet = re.sub(r"NASAHurricane", "NASA Hurricane", tweet)
    tweet = re.sub(r"onlinecommunities", "online communities", tweet)
    tweet = re.sub(r"humanconsumption", "human consumption", tweet)
    tweet = re.sub(r"Typhoon-Devastated", "Typhoon Devastated", tweet)
    tweet = re.sub(r"Meat-Loving", "Meat Loving", tweet)
    tweet = re.sub(r"facialabuse", "facial abuse", tweet)
    tweet = re.sub(r"LakeCounty", "Lake County", tweet)
    tweet = re.sub(r"BeingAuthor", "Being Author", tweet)
    tweet = re.sub(r"withheavenly", "with heavenly", tweet)
    tweet = re.sub(r"thankU", "thank you", tweet)
    tweet = re.sub(r"iTunesMusic", "iTunes Music", tweet)
    tweet = re.sub(r"OffensiveContent", "Offensive Content", tweet)
    tweet = re.sub(r"WorstSummerJob", "Worst Summer Job", tweet)
    tweet = re.sub(r"HarryBeCareful", "Harry Be Careful", tweet)
    tweet = re.sub(r"NASASolarSystem", "NASA Solar System", tweet)
    
    return tweet

df_train['text_cleaned'] = df_train['text'].apply(lambda x : clean(x))
df_test['text_cleaned'] = df_test['text'].apply(lambda x : clean(x))

train_tweets_cleaned = df_train['text_cleaned'].apply(lambda s: s.split()).values
train_vocab_cleaned = build_vocab(train_tweets_cleaned)
test_tweets_cleaned = df_test['text_cleaned'].apply(lambda s: s.split()).values
test_vocab_cleaned = build_vocab(test_tweets_cleaned)

train_oov_glove = check_coverage(train_vocab_cleaned, embeddings_glove, 'GloVe', 'Training')
test_oov_glove = check_coverage(test_vocab_cleaned, embeddings_glove, 'GloVe', 'Test')
train_oov_fasttext = check_coverage(train_vocab_cleaned, embeddings_fasttext, 'FastText', 'Training')
test_oov_fasttext = check_coverage(test_vocab_cleaned, embeddings_fasttext, 'FastText', 'Test')

## **5. Test Set Labels**
Test set labels can be found on [this](https://www.figure-eight.com/data-for-everyone/) website. Dataset is named **Disasters on social media**. This is how people are submitting perfect scores. Other "Getting Started" competitions also have their test labels leaked. The main point of "Getting Started" competitions is **learning and sharing**, and perfect score doesn't mean anything. 

> **Phil Culliton wrote:**
> For the AutoML prize, any use of the label set will result in disqualification.

According to [@philculliton](https://www.kaggle.com/philculliton) from Kaggle Team, competitors who use test set labels in any way are not eligible to win AutoML prize. There are no other penalties for using them.

In [ ]:
df_leak = pd.read_csv('../input/nlp-with-disaster-tweets-test-set-with-labels/socialmedia-disaster-tweets-DFE.csv', encoding ='ISO-8859-1')[['choose_one', 'text']]

# Creating target and id
df_leak['target'] = (df_leak['choose_one'] == 'Relevant').astype(np.int8)
df_leak['id'] = df_leak.index.astype(np.int16)
df_leak.drop(columns=['choose_one', 'text'], inplace=True)

# Merging target to test set
df_test = df_test.merge(df_leak, on=['id'], how='left')

print('Leaked Data Set Shape = {}'.format(df_leak.shape))
print('Leaked Data Set Memory Usage = {:.2f} MB'.format(df_leak.memory_usage().sum() / 1024**2))

In [ ]:
ORIGINAL_FEATURES = ['id', 'keyword', 'location', 'text', 'target', 'text_cleaned']

df_train[ORIGINAL_FEATURES].to_pickle('train.pkl')
df_test[ORIGINAL_FEATURES].to_pickle('test.pkl')

submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
submission['target'] = df_test['target'].values
submission.to_csv('submission.csv', index=False)

print('Training Set Shape = {}'.format(df_train[ORIGINAL_FEATURES].shape))
print('Training Set Memory Usage = {:.2f} MB'.format(df_train[ORIGINAL_FEATURES].memory_usage().sum() / 1024**2))
print('Test Set Shape = {}'.format(df_test[ORIGINAL_FEATURES].shape))
print('Test Set Memory Usage = {:.2f} MB'.format(df_test[ORIGINAL_FEATURES].memory_usage().sum() / 1024**2))